In [29]:
%matplotlib inline

import os
import time
import copy
import pandas as pd
import numpy as np

from random import seed
from random import randint
import random

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

from PIL import Image
from matplotlib import pyplot as plt

from sklearn.metrics import fbeta_score, precision_recall_fscore_support
from sklearn import metrics

import warnings
warnings.filterwarnings("ignore")

torch.set_num_threads(5)

from tqdm import tqdm_notebook as tqdm

if not os.path.exists('Features'):
    os.makedirs('Features')

"""
voxel_type can take 4 values - None, bp, cc, mf
"""

voxel_type = 'None'
train_dir = os.path.join('..',f'pictures/train/2/{voxel_type}')
labels_csv= os.path.join('../labels/struct','new_2.csv')
resnet_weights_path = 'Resnet50.pth'
weights_path = f'weights_{voxel_type}.pth'

In [5]:
def seed_all(seed=27):
    """https://pytorch.org/docs/stable/notes/randomness.html"""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [6]:
BATCH_SIZE = 50
NUM_EPOCHS = 20
PERCENTILE = 99.7
LEARNING_RATE = 0.00001
DISABLE_TQDM = True

In [7]:
device = torch.device("cuda:6" if torch.cuda.is_available() else "cpu")
print(device)

cuda:6


In [8]:
df = pd.read_csv(labels_csv)
attribute_dict = dict(zip(df.accession_no,df.labels))
output_dim = 439
# del df,labels_csv

In [9]:
# df = pd.read_csv(train_csv)
# labels_dict = dict(zip(df.id,df.attribute_ids))

In [10]:
# need to add more transforms here
data_transforms = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
    ])

In [11]:
from torch.utils import data
class ImageData(data.Dataset):
    def __init__(self,df,dirpath,transform,test = False):
        self.df = df
        self.test = test
        self.dirpath = dirpath
        self.conv_to_tensor = transform
        #image data 
        if not self.test:
            self.image_arr = np.asarray(str(self.dirpath)+'/'+self.df.iloc[:, 0])
        else:
            self.image_arr = np.asarray(str(self.dirpath)+'/'+self.df.iloc[:, 0])
        
        #labels data
        if not self.test:
             self.label_df = self.df.iloc[:,1]
        
        # Calculate length of df
        self.data_len = len(self.df.index)

    def __len__(self):
        return self.data_len
    
    def __getitem__(self, idx):
        image_name = self.image_arr[idx]
        img = Image.open(image_name)
        img = img.convert(mode = 'RGB')
        img_tensor = self.conv_to_tensor(img)
        if not self.test:
            image_labels = self.label_df[idx]
            label_tensor = torch.zeros((1, output_dim))
            image_labels = [int(x) for x in image_labels.split(',')]
            for i, label in enumerate(image_labels):
                label_tensor[0, i] = label
            image_label = torch.tensor(label_tensor,dtype= torch.float32)
            return (img_tensor,image_label.squeeze())
        return (img_tensor)

In [12]:
df = pd.read_csv(labels_csv)

from sklearn.model_selection import train_test_split
train_df,val_df = train_test_split(df, test_size=0.20)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
print(f"Validation_Data Length: {len(val_df)}\n Train_Data Length: {len(train_df)}")

Validation_Data Length: 2064
 Train_Data Length: 8255


In [13]:

# Train dataset
train_dataset = ImageData(train_df,train_dir,data_transforms)
train_loader = data.DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,shuffle=False)

# validation dataset
val_dataset = ImageData(val_df,train_dir,data_transforms)
val_loader = data.DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,shuffle=False)

dataloaders_dict = {'train':train_loader, 'val':val_loader}

In [14]:
features, labels = next(iter(train_loader))
print(f'Train Features: {features.shape}\nTrain Labels: {labels.shape}')
print()
features, labels = next(iter(val_loader))
print(f'Validation Features: {features.shape}\nValidation Labels: {labels.shape}')
print()

Train Features: torch.Size([50, 3, 224, 224])
Train Labels: torch.Size([50, 439])

Validation Features: torch.Size([50, 3, 224, 224])
Validation Labels: torch.Size([50, 439])



In [15]:
resnet_cls = models.resnet50()
resnet_cls.load_state_dict(torch.load(resnet_weights_path))

class AvgPool(nn.Module):
    def forward(self, x):
        return F.avg_pool2d(x, x.shape[2:])
    
class ResNet50(nn.Module):
    def __init__(self,num_outputs):
        super(ResNet50,self).__init__()
        self.resnet = resnet_cls
        layer4 = self.resnet.layer4
        self.resnet.layer4 = nn.Sequential(nn.Dropout(0.5), layer4)
        self.resnet.avgpool = AvgPool()
        self.resnet.fc = nn.Linear(2048, 1024)
        
        self.fc1 = nn.Linear(1024, 256)
        self.fc2 = nn.Linear(256, num_outputs)
        self.bn1 = nn.BatchNorm1d(1024)

        for param in self.resnet.parameters():
            param.requires_grad = False

        for param in self.resnet.layer4.parameters():
            param.requires_grad = True

        for param in self.resnet.fc.parameters():
            param.requires_grad = True
            
        for param in self.fc1.parameters():
            param.requires_grad = True
            
        for param in self.bn1.parameters():
            param.requires_grad = True
            
        for param in self.fc2.parameters():
            param.requires_grad = True

#         for param in self.resnet.classifier.parameters():
#             param.requires_grad = True
            
    def forward(self,x):
        out = self.bn1(self.resnet(x))
        out = F.sigmoid(self.fc1(out))
        out = self.fc2(out)
        return out
    
NeuralNet = ResNet50(num_outputs = output_dim)

In [16]:
NeuralNet

ResNet50(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
          (0): Conv2

In [17]:
total_params = sum(p.numel() for p in NeuralNet.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(p.numel() for p in NeuralNet.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

25,983,479 total parameters.
17,440,183 training parameters.


In [18]:
print("TRAINING")
print("training examples: ",len(train_dataset))
print("batch size: ",BATCH_SIZE)
print("batches available: ",len(train_loader))
print()
print("VALIDATION")
print("validation examples: ",len(val_dataset))
print("batch size: ",BATCH_SIZE)
print("batches available: ",len(val_loader))
print()

TRAINING
training examples:  8255
batch size:  50
batches available:  166

VALIDATION
validation examples:  2064
batch size:  50
batches available:  42



In [19]:
NeuralNet = NeuralNet.to(device)
optimizer = optim.Adam(NeuralNet.parameters(),lr = LEARNING_RATE)
loss_func = torch.nn.BCEWithLogitsLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience = 2)
best_loss = np.inf
best_f_score = np.inf
best_precision = np.inf
best_recall = np.inf

In [20]:
def f2_score(y_true, y_pred, threshold=0.5):
    return fbeta_score(y_true, y_pred, 2, threshold)


def fbeta_score(y_true, y_pred, beta, threshold, eps=1e-9):
    beta2 = beta**2

    y_pred = torch.ge(y_pred.float(), threshold).float()
    y_true = y_true.float()

    true_positive = (y_pred * y_true).sum(dim=1)
    precision = true_positive.div(y_pred.sum(dim=1).add(eps))
    recall = true_positive.div(y_true.sum(dim=1).add(eps))

    return torch.mean(
        (precision*recall).
        div(precision.mul(beta2) + recall + eps).
        mul(1 + beta2))

In [21]:
#kaggle

def F_score(logit, label, threshold=0.5, beta=2):
    prob = torch.sigmoid(logit)
    prob = prob > threshold
    label = label > threshold

    TP = (prob & label).sum(1).float()
    TN = ((~prob) & (~label)).sum(1).float()
    FP = (prob & (~label)).sum(1).float()
    FN = ((~prob) & label).sum(1).float()

    precision = TP / (TP + FP + 1e-12)
    recall = TP / (TP + FN + 1e-12)
    F2 = (1 + beta**2) * precision * recall / (beta**2 * precision + recall + 1e-12)
    return F2.mean(0)

In [22]:
pre_per_epoch = []
recall_per_epoch = []
f_per_epoch = []
loss_per_epoch = []

for epoch in range(NUM_EPOCHS):
    for phase in ['train', 'val']:
        start_time = time.time()
        if phase == 'train':
            NeuralNet.train()
        else:
            NeuralNet.eval()
        
        running_loss = 0.0
        running_f_score = 0.0
        running_precision = 0.0
        running_recall = 0.0
        
        
        for images_batch, labels_batch in tqdm(dataloaders_dict[phase],disable = DISABLE_TQDM):
            images_batch = images_batch.to(device)
            labels_batch = labels_batch.to(device)
            
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(phase == 'train'):
                pred_batch = NeuralNet(images_batch)
                _, preds = torch.max(pred_batch.data, 1)
                loss = loss_func(pred_batch,labels_batch)
            if phase == 'train':
                loss.backward()
                optimizer.step()
                
            labels_cpu = labels_batch.cpu().detach().numpy()
            pred_cpu = pred_batch.cpu().detach().numpy()
            
#             print(metrics.multilabel_confusion_matrix(labels_cpu, pred_cpu, samplewise = true))
            
            running_loss += loss.item() * images_batch.size(0)
#             running_f2_loss += F_score(labels_batch, pred_batch)
            temp_precision, temp_recall, temp_f_score, _ = precision_recall_fscore_support(
                                                            labels_cpu, pred_cpu > 0.5, beta=2, average='samples')
        
            running_precision += (temp_precision * len(images_batch))
            running_recall += (temp_recall * len(images_batch))
            running_f_score += (temp_f_score * len(images_batch))
            
            pre_per_epoch.append(temp_precision)
            recall_per_epoch.append(temp_recall)
            f_per_epoch.append(temp_f_score)
            loss_per_epoch.append(loss.item())
            
        epoch_loss = running_loss / len(dataloaders_dict[phase].dataset)
        epoch_f_score = running_f_score / len(dataloaders_dict[phase].dataset)
        epoch_precision = running_precision / len(dataloaders_dict[phase].dataset)
        epoch_recall = running_recall / len(dataloaders_dict[phase].dataset)
        
        if phase == 'val' and epoch_loss < best_loss:
#             print("model val_loss Improved from {:.8f} to {:.8f}".format(best_loss,epoch_loss))
            best_loss = epoch_loss
            best_model_wts = copy.deepcopy(NeuralNet.state_dict())
            torch.save(NeuralNet.state_dict(), weights_path)
        
        if phase == 'val':
            scheduler.step(epoch_loss)
        
        elapsed_time = time.time()-start_time
        print("Phase: {} | Epoch: {}/{} | {}_loss:{:.8f} | f_score:{:.8f} | precision:{:.8f} | recall:{:.8f} | Time: {:.4f}s".format(phase,
                                                                              epoch+1,
                                                                              NUM_EPOCHS,
                                                                              phase,
                                                                              epoch_loss,
                                                                              epoch_f_score,
                                                                              epoch_precision,
                                                                              epoch_recall,
                                                                              elapsed_time))
NeuralNet.load_state_dict(torch.load(best_model_wts))

Phase: train | Epoch: 1/20 | train_loss:0.66014285 | f_score:0.01992434 | precision:0.02757959 | recall:0.02205206 | Time: 348.6563s
Phase: val | Epoch: 1/20 | val_loss:0.60991658 | f_score:0.00654550 | precision:0.01705984 | recall:0.00587375 | Time: 88.6514s
Phase: train | Epoch: 2/20 | train_loss:0.56629287 | f_score:0.06781350 | precision:0.29129662 | recall:0.05881499 | Time: 166.9354s
Phase: val | Epoch: 2/20 | val_loss:0.51786461 | f_score:0.13845929 | precision:0.86224160 | recall:0.11667046 | Time: 43.8098s
Phase: train | Epoch: 3/20 | train_loss:0.47883489 | f_score:0.16349665 | precision:0.87960024 | recall:0.13938395 | Time: 170.2129s
Phase: val | Epoch: 3/20 | val_loss:0.43409168 | f_score:0.22067152 | precision:0.84883721 | recall:0.19096282 | Time: 41.2386s
Phase: train | Epoch: 4/20 | train_loss:0.40117185 | f_score:0.22358684 | precision:0.84185342 | recall:0.19419805 | Time: 169.3039s
Phase: val | Epoch: 4/20 | val_loss:0.36362798 | f_score:0.22319096 | precision:0.84

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [23]:
! nvidia-smi

Fri Jun 21 21:02:28 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.111                Driver Version: 384.111                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0  On |                  N/A |
|  0%   32C    P8    18W / 250W |    787MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:05:00.0 Off |                  N/A |
| 25%   45C    P2    56W / 250W |   3706MiB / 11172MiB |      2%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [24]:
resnet_cls = models.resnet50()
resnet_cls.load_state_dict(torch.load(resnet_weights_path))

class AvgPool(nn.Module):
    def forward(self, x):
        return F.avg_pool2d(x, x.shape[2:])
    
class ResNet50(nn.Module):
    def __init__(self,num_outputs):
        super(ResNet50,self).__init__()
        self.resnet = resnet_cls
        layer4 = self.resnet.layer4
        self.resnet.layer4 = nn.Sequential(nn.Dropout(0.5), layer4)
        self.resnet.avgpool = AvgPool()
        self.resnet.fc = nn.Linear(2048, 1024)
        
        self.fc1 = nn.Linear(1024, 256)
        self.fc2 = nn.Linear(256, num_outputs)
        self.bn1 = nn.BatchNorm1d(1024)

        for param in self.resnet.parameters():
            param.requires_grad = False

        for param in self.resnet.layer4.parameters():
            param.requires_grad = True

        for param in self.resnet.fc.parameters():
            param.requires_grad = True
            
        for param in self.fc1.parameters():
            param.requires_grad = True
            
        for param in self.bn1.parameters():
            param.requires_grad = True
            
        for param in self.fc2.parameters():
            param.requires_grad = True

#         for param in self.resnet.classifier.parameters():
#             param.requires_grad = True
            
    def forward(self,x):
        out = self.bn1(self.resnet(x))
        out = F.sigmoid(self.fc1(out))
        out = self.fc2(out)
        return out
    
NeuralNet = ResNet50(num_outputs = output_dim)
NeuralNet.load_state_dict(torch.load(weights_path))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [25]:
newmodel = torch.nn.Sequential(*(list(NeuralNet.children())[:-2]))
print(newmodel)

Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
          (0): Conv2d(6

In [30]:
features = []
acc = []

for img_name in tqdm(os.listdir(train_dir)):
    img = Image.open(train_dir + '/' + img_name)
    img = img.convert(mode = 'RGB')
    b = transforms.ToTensor()
    img_tensor = b(img)
    img_tensor = img_tensor.unsqueeze(0)
    pred = newmodel(img_tensor)
    temp = pred.detach().numpy()
    features.append(temp)
    acc.append(img_name.strip().split('_')[0])
res_df = pd.DataFrame({'accession': acc,'features':features})

res_df.to_pickle(f'Features/features_{voxel_type}.pkl')